# Keyed dependency injection services

- The `IKeyedServiceProvider` interface.
- The `ServiceKeyAttribute` attribute, which can be used to inject the key that was used for registration/resolution in the constructor.
- The `FromKeyedServicesAttribute` attribute, which can be used on service constructor parameters to specify which keyed service to use.
- Various new extension methods for `IServiceCollection` to support keyed services, for example, `ServiceCollectionServiceExtensions.AddKeyedScoped`.
- The `ServiceProvider` implementation of `IKeyedServiceProvider`.

In [ ]:
#r "nuget:Microsoft.DotNet.Interactive.AspNetCore, *-*"

In [ ]:
#!aspnet
using Microsoft.Extensions.DependencyInjection;

var builder = WebApplication.CreateBuilder();

builder.Services.AddSingleton<BigCacheConsumer>();
builder.Services.AddSingleton<SmallCacheConsumer>();
builder.Services.AddKeyedSingleton<ICache, BigCache>("big");
builder.Services.AddKeyedSingleton<ICache, SmallCache>("small");

var app = builder.Build();

app.MapGet("/big", (BigCacheConsumer data) => data.GetData());
app.MapGet("/small", (SmallCacheConsumer data) => data.GetData());
app.MapGet("/big-cache", ([FromKeyedServices("big")] ICache cache) => cache.Get("data"));
app.MapGet("/small-cache", (HttpContext httpContext) => httpContext.RequestServices.GetRequiredKeyedService<ICache>("small").Get("data"));

app.Run();

class BigCacheConsumer([FromKeyedServices("big")] ICache cache)
{
    public object? GetData() => cache.Get("data");
}

class SmallCacheConsumer(IServiceProvider serviceProvider)
{
    public object? GetData() => serviceProvider.GetRequiredKeyedService<ICache>("small").Get("data");
}

public interface ICache
{
    object Get(string key);
}

public class BigCache : ICache
{
    public object Get(string key) => $"Resolving {key} from big cache.";
}

public class SmallCache : ICache
{
    public object Get(string key) => $"Resolving {key} from small cache.";
}

## Real world example

In [ ]:
public class User {}

public interface IUserService {
    Task<User> GetUser(string userId);
    Task<IEnumerable<User>> GetUsers();
}

public interface IUserServiceAsUser : IUserService { }
public interface IUserServiceAsApp : IUserService { }

public class AADUserService : IUserService, IUserServiceAsUser, IUserServiceAsApp { }

services.AddScoped<IUserServiceAsApp>(sp => new AADUserService(/* app specific login service*/));
services.AddScoped<IUserServiceAsUser>(sp => new AADUserService(/* user specific login service*/));

// with keyed DI
builder.Services.AddKeyedScoped<IUserService>("app", sp => new AADUserService(/* app specific login service*/)));
builder.Services.AddKeyedScoped<IUserService>("user", sp => new AADUserService(/* user specific login service*/));

public void GetUsers([FromKeyedServices("app")] IUserService userService) { } 
public void GetUsers([FromKeyedServices("user")] IUserService userService) { } 